In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import math
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm

In [2]:
df = pd.read_csv('C:\\Users\\DHAN RAJ\\Downloads\\btp_forecast_preprocessed_data\\kolkata\\hourly\\imputed_hourly_kolkata_data.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
data = df.drop(columns=['Timestamp'])
# Assuming df is your DataFrame
correlation_matrix = data.corr()
print(correlation_matrix)

                     PM2.5 (µg/m³)  PM10 (µg/m³)  NO (µg/m³)  NO2 (µg/m³)  \
PM2.5 (µg/m³)             1.000000      0.945895    0.402924     0.578399   
PM10 (µg/m³)              0.945895      1.000000    0.415400     0.602774   
NO (µg/m³)                0.402924      0.415400    1.000000     0.589346   
NO2 (µg/m³)               0.578399      0.602774    0.589346     1.000000   
NOx (ppb)                 0.541315      0.563648    0.860616     0.870930   
NH3 (µg/m³)               0.305191      0.315169    0.196940     0.259006   
SO2 (µg/m³)               0.329998      0.354382    0.084223     0.332977   
CO (mg/m³)                0.642284      0.649653    0.636311     0.634489   
Ozone (µg/m³)             0.030328      0.073840   -0.195621    -0.138651   
Benzene (µg/m³)           0.502288      0.513828    0.430681     0.512070   
Toluene (µg/m³)           0.496355      0.500900    0.439660     0.545152   
Eth-Benzene (µg/m³)       0.415017      0.429584    0.301482     0.351807   

In [24]:
# Set correlation threshold
threshold = 0.8

# Find pairs with correlation above threshold
highly_correlated_pairs = [(column1, 'PM2.5 (µg/m³)') for column1 in correlation_matrix.columns if abs(correlation_matrix[column1]['PM2.5 (µg/m³)']) > threshold and column1 != 'PM2.5 (µg/m³)']

In [25]:
from statsmodels.tsa.stattools import grangercausalitytests

# Apply Granger's Causality Test
for pair in highly_correlated_pairs:
    print()
    print(f"Granger Causality Test for pair {pair}:")
    granger_test_result = grangercausalitytests(df[list(pair)], maxlag=12)


Granger Causality Test for pair ('PM10 (µg/m³)', 'PM2.5 (µg/m³)'):

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=667.3604, p=0.0000  , df_denom=26276, df_num=1
ssr based chi2 test:   chi2=667.4366, p=0.0000  , df=1
likelihood ratio test: chi2=659.1016, p=0.0000  , df=1
parameter F test:         F=667.3604, p=0.0000  , df_denom=26276, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=455.4433, p=0.0000  , df_denom=26273, df_num=2
ssr based chi2 test:   chi2=911.0600, p=0.0000  , df=2
likelihood ratio test: chi2=895.6225, p=0.0000  , df=2
parameter F test:         F=455.4433, p=0.0000  , df_denom=26273, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=326.6176, p=0.0000  , df_denom=26270, df_num=3
ssr based chi2 test:   chi2=980.1140, p=0.0000  , df=3
likelihood ratio test: chi2=962.2774, p=0.0000  , df=3
parameter F test:         F=326.6176, p=0.0000  , df_denom=26270, df_num=3

Granger Causa

In [26]:
cols = [i[0] for i in highly_correlated_pairs]

In [27]:
cols.append('PM2.5 (µg/m³)')

In [28]:
cols

['PM10 (µg/m³)', 'PM2.5 (µg/m³)']

In [29]:
df_corr = df[cols]
df_corr.index = df['Timestamp']
from statsmodels.tsa.vector_ar.vecm import coint_johansen
johan_test_temp = df_corr
coint_johansen(johan_test_temp,-1,1).eig

array([0.09568381, 0.00689105])

In [30]:
colums = [i.split()[0] for i in cols]
val = pd.read_excel("C:/Users/DHAN RAJ/Downloads/kolkata_hourly_Jan_1.xlsx")
val['Timestamp'] = pd.to_datetime(val['Timestamp'])
val.set_index('Timestamp', inplace=True)
val = val[colums]
val.columns = cols
val

,PM10 (µg/m³),PM2.5 (µg/m³)
Timestamp,,
2024-01-01 00:00:00,202.85,114.00
2024-01-01 01:00:00,198.10,120.10
2024-01-01 02:00:00,189.50,114.50
2024-01-01 03:00:00,180.95,106.20
2024-01-01 04:00:00,176.30,102.60
2024-01-01 05:00:00,NaN,95.75
2024-01-01 06:00:00,NaN,88.00
2024-01-01 07:00:00,NaN,86.70
2024-01-01 08:00:00,NaN,94.80


In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
# scaling dataset
scaled_train = scaler.fit_transform(df_corr)

# Normalizing values between 0 and 1
scaled_test = scaler.transform(val)

In [32]:
#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=scaled_train)
model_fit = model.fit()

model_fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 27, Apr, 2024
Time:                     18:38:25
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -15.2828
Nobs:                     26279.0    HQIC:                  -15.2841
Log likelihood:           126263.    FPE:                2.30115e-07
AIC:                     -15.2847    Det(Omega_mle):     2.30062e-07
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.006023         0.000257           23.448           0.000
L1.y1         0.813134         0.005703          142.586           0.000
L1.y2         0.121509         0.004704           25.833           0.000

Results for equation 

In [33]:
# make prediction on validation
lag_order = model_fit.k_ar
prediction = model_fit.forecast(scaled_train[-lag_order:], steps=24)

In [34]:
prediction

array([[0.21471443, 0.24672711],
       [0.2105945 , 0.24316149],
       [0.20681118, 0.23953298],
       [0.20329395, 0.23589681],
       [0.19999214, 0.23229067],
       [0.19686915, 0.22873997],
       [0.1938983 , 0.22526147],
       [0.19105993, 0.22186591],
       [0.18833936, 0.21855979],
       [0.18572545, 0.21534667],
       [0.18320958, 0.2122281 ],
       [0.18078489, 0.20920418],
       [0.17844587, 0.2062741 ],
       [0.1761879 , 0.20343637],
       [0.17400706, 0.20068909],
       [0.17189993, 0.19803008],
       [0.16986345, 0.19545702],
       [0.16789487, 0.19296747],
       [0.16599165, 0.19055897],
       [0.16415143, 0.18822906],
       [0.16237197, 0.1859753 ],
       [0.16065118, 0.18379528],
       [0.15898706, 0.18168665],
       [0.15737769, 0.17964711]])

In [35]:
scaled_test

array([[0.21675705, 0.2573521 ],
       [0.21160521, 0.27122615],
       [0.20227766, 0.25848932],
       [0.19300434, 0.23961153],
       [0.18796095, 0.23142357],
       [       nan, 0.2158437 ],
       [       nan, 0.19821684],
       [       nan, 0.19526008],
       [       nan, 0.21368299],
       [       nan, 0.23392544],
       [       nan, 0.22914913],
       [       nan, 0.22187095],
       [0.22483731, 0.24427411],
       [0.22483731, 0.26895171],
       [       nan,        nan],
       [0.21505423, 0.22864876],
       [0.19058568, 0.21300066],
       [0.19037961, 0.20356176],
       [0.17834056, 0.20453977],
       [0.20260304, 0.22603316],
       [0.20818872, 0.23797393],
       [0.20759219, 0.23942957],
       [0.2095987 , 0.24404667],
       [0.22885033, 0.28714718]])

In [36]:
val

,PM10 (µg/m³),PM2.5 (µg/m³)
Timestamp,,
2024-01-01 00:00:00,202.85,114.00
2024-01-01 01:00:00,198.10,120.10
2024-01-01 02:00:00,189.50,114.50
2024-01-01 03:00:00,180.95,106.20
2024-01-01 04:00:00,176.30,102.60
2024-01-01 05:00:00,NaN,95.75
2024-01-01 06:00:00,NaN,88.00
2024-01-01 07:00:00,NaN,86.70
2024-01-01 08:00:00,NaN,94.80


In [37]:
preds = scaler.inverse_transform(prediction)
preds

array([[200.96670033, 109.32851061],
       [197.16812458, 107.76081146],
       [193.67991196, 106.16546604],
       [190.43702178, 104.56675027],
       [187.39275291, 102.9812398 ],
       [184.51335463, 101.42010213],
       [181.77422864,  99.89071113],
       [179.15725212,  98.39778389],
       [176.64889075,  96.94418136],
       [174.23886827,  95.53147183],
       [171.91922835,  94.16032748],
       [169.68367266,  92.83080336],
       [167.52709387,  91.54253339],
       [165.44524574,  90.29486809],
       [163.43451002,  89.08697114],
       [161.49173148,  87.91788701],
       [159.61410104,  86.78658819],
       [157.79907269,  85.6920081 ],
       [156.04430444,  84.63306386],
       [154.347616  ,  83.608672  ],
       [152.70695842,  82.61775914],
       [151.12039213,  81.65926917],
       [149.58607091,  80.73216804],
       [148.10223003,  79.8354467 ]])

In [38]:
val['PM2.5 (µg/m³)'].iloc[14] = sum(val['PM2.5 (µg/m³)'].iloc[9:14])/5

C:\Users\DHAN RAJ\AppData\Local\Temp\ipykernel_21368\3879252357.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  val['PM2.5 (µg/m³)'].iloc[14] = sum(val['PM2.5 (µg/m³)'].iloc[9:14])/5


In [39]:
mae = np.abs(preds[:,len(cols)-1] - val['PM2.5 (µg/m³)']).mean()

In [40]:
mae

13.049276767680476

In [43]:
df_corr.values.shape

(26280, 2)

In [53]:
from statsmodels.tsa.statespace.varmax import VARMAX

# Fit VARMA model
model = VARMAX(df_corr.values, order=(2, 2))
results = model.fit()

# Forecast
forecast = results.forecast(steps=24)
mae = np.abs(forecast[:,len(cols)-1] - val['PM2.5 (µg/m³)']).mean()
mae

C:\Users\DHAN RAJ\miniconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\DHAN RAJ\miniconda3\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


12.382484282736542

In [49]:
from statsmodels.tsa.vector_ar.vecm import VECM

# Fit VECM model
model = VECM(df_corr.values)
results = model.fit()

# Forecast
forecast = results.predict(steps=24)


In [50]:
mae = np.abs(forecast[:,len(cols)-1] - val['PM2.5 (µg/m³)']).mean()

In [51]:
mae

10.411376627182804